In [1]:
!pip install -q sentence-transformers rouge transformers evaluate bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 46.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from sentence_transformers import CrossEncoder
from evaluate import load

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
nltk.download('wordnet')

In [5]:
# Define root_path and data_path

In [ ]:
# Vector

# Prepare to accumulate DataFrames
dfs = []

# Loop through the files in the specified directory
for i in range(1, 6):  # Assuming files are numbered from 1 to 10
    file_name = f'file_name'
    file_path = os.path.join(data_path, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print(f'File not found: {file_name}')

# Concatenate all DataFrames in the list into a single DataFrame
vector_df = pd.concat(dfs, ignore_index=True)

print(vector_df.shape)

In [ ]:
vector_df_without_column = vector_df.drop(columns=['teacher_answer'])
vector_df_without_column_replaced = vector_df_without_column.replace('None', pd.NA)
len(vector_df_without_column_replaced.dropna())

In [ ]:
# Graph

# Prepare to accumulate DataFrames
dfs = []

# Loop through the files in the specified directory
for i in range(1, 5):  # Assuming files are numbered from 1 to 10
    file_name = f'05132024-results-graph-llama3_{i}.csv'
    file_path = os.path.join(data_path, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print(f'File not found: {file_name}')

# Concatenate all DataFrames in the list into a single DataFrame
graph_df = pd.concat(dfs, ignore_index=True)

print(graph_df.shape)

In [ ]:
graph_df_without_column = graph_df.drop(columns=['teacher_answer'])
graph_df_without_column_replaced = graph_df_without_column.replace('None', pd.NA)
len(graph_df_without_column_replaced.dropna())

In [10]:
def sample_nan_or_none_row(df):
    """
    Randomly samples a row from the DataFrame that contains at least one NaN or the string "None",
    without altering the original DataFrame values.

    Parameters:
    - df: pandas.DataFrame to sample from.

    Returns:
    - A randomly sampled row (as a DataFrame) that contains at least one NaN or "None".
    """
    # Create a mask to identify rows with at least one occurrence of np.nan or "None"
    mask = df.apply(lambda x: x.isna() | (x == "None")).any(axis=1)

    # Filter the DataFrame to only include such rows
    df_with_nan_or_none = df[mask]

    # Randomly sample one row from this filtered DataFrame
    if not df_with_nan_or_none.empty:
        return df_with_nan_or_none.sample(n=1)
    else:
        return None  # Or return an empty DataFrame, depending on your preference

In [ ]:
# First, merge vector_df with graph_df
merged_df_all = pd.merge(vector_df, graph_df.drop(columns=['question', 'teacher_answer']), on='id', how='left')
merged_df_all.shape

In [12]:
merged_df_all.columns

Index(['id', 'question', 'teacher_answer', 'response_vector_no_guidance',
       'context_vector_no_guidance', 'response_vector_low_guidance',
       'context_vector_low_guidance', 'response_vector_high_guidance',
       'context_vector_high_guidance', 'response_graph_no_guidance',
       'context_graph_no_guidance', 'response_graph_low_guidance',
       'context_graph_low_guidance', 'response_graph_high_guidance',
       'context_graph_high_guidance'],
      dtype='object')

In [13]:
for col in merged_df_all.columns:
    if col != 'id':
        merged_df_all[col] = merged_df_all[col].astype(str)

In [14]:
bert_score = load("bertscore")

In [ ]:
model = CrossEncoder('vectara/hallucination_evaluation_model')

In [ ]:
metric_columns = [
    # Vector
    # no guidance reference
    'bleu_score_vector_no_guidance_reference',
    'bert_score_f_vector_no_guidance_reference',
    'cross_encoder_score_vector_no_guidance_reference',
    # low guidance reference
    'bleu_score_vector_low_guidance_reference',
    'bert_score_f_vector_low_guidance_reference',
    'cross_encoder_score_vector_low_guidance_reference',
    # high guidance reference
    'bleu_score_vector_high_guidance_reference',
    'bert_score_f_vector_high_guidance_reference',
    'cross_encoder_score_vector_high_guidance_reference',
    # no guidance context
    'bleu_score_vector_no_guidance_context',
    'bert_score_f_vector_no_guidance_context',
    'cross_encoder_score_vector_no_guidance_context',
    # low guidance context
    'bleu_score_vector_low_guidance_context',
    'bert_score_f_vector_low_guidance_context',
    'cross_encoder_score_vector_low_guidance_context',
    # high guidance context
    'bleu_score_vector_high_guidance_context',
    'bert_score_f_vector_high_guidance_context',
    'cross_encoder_score_vector_high_guidance_context',
    # Graph
    # no guidance reference
    'bleu_score_graph_no_guidance_reference',
    'bert_score_f_graph_no_guidance_reference',
    'cross_encoder_score_graph_no_guidance_reference',
    # low guidance reference
    'bleu_score_graph_low_guidance_reference',
    'bert_score_f_graph_low_guidance_reference',
    'cross_encoder_score_graph_low_guidance_reference',
    # high guidance reference
    'bleu_score_graph_high_guidance_reference',
    'bert_score_f_graph_high_guidance_reference',
    'cross_encoder_score_graph_high_guidance_reference',
    # no guidance context
    'bleu_score_graph_no_guidance_context',
    'bert_score_f_graph_no_guidance_context',
    'cross_encoder_score_graph_no_guidance_context',
    # low guidance context
    'bleu_score_graph_low_guidance_context',
    'bert_score_f_graph_low_guidance_context',
    'cross_encoder_score_graph_low_guidance_context',
    # high guidance context
    'bleu_score_graph_high_guidance_context',
    'bert_score_f_graph_high_guidance_context',
    'cross_encoder_score_graph_high_guidance_context',
]

metrics_results = {metric: [] for metric in metric_columns}

for i, row in merged_df_all.iterrows():
    if i % 10 == 0:
        print(f'i: {i}')

    question = row['question']
    teacher_answer = row['teacher_answer']
    # Vector
    response_vector_no_guidance = row['response_vector_no_guidance']
    context_vector_no_guidance = row['context_vector_no_guidance']
    response_vector_low_guidance = row['response_vector_low_guidance']
    context_vector_low_guidance = row['context_vector_low_guidance']
    response_vector_high_guidance = row['response_vector_high_guidance']
    context_vector_high_guidance = row['context_vector_high_guidance']
    # Graph
    response_graph_no_guidance = row['response_graph_no_guidance']
    context_graph_no_guidance = row['context_graph_no_guidance']
    response_graph_low_guidance = row['response_graph_low_guidance']
    context_graph_low_guidance = row['context_graph_low_guidance']
    response_graph_high_guidance = row['response_graph_high_guidance']
    context_graph_high_guidance = row['context_graph_high_guidance']

    if not pd.isna(teacher_answer):
        # Vector

        # No guidance.
        bleu_score_vector_no_guidance_reference = sentence_bleu([teacher_answer.split()], response_vector_no_guidance.split())

        bert_score_result_vector_no_guidance_reference = bert_score.compute(predictions=[response_vector_no_guidance], references=[teacher_answer], lang="en")
        bert_score_f_vector_no_guidance_reference = bert_score_result_vector_no_guidance_reference['f1'][0]

        cross_encoder_score_vector_no_guidance_reference = model.predict([[teacher_answer, response_vector_no_guidance]])[0]

        # Low guidance.
        bleu_score_vector_low_guidance_reference = sentence_bleu([teacher_answer.split()], response_vector_low_guidance.split())

        bert_score_result_vector_low_guidance_reference = bert_score.compute(predictions=[response_vector_low_guidance], references=[teacher_answer], lang="en")
        bert_score_f_vector_low_guidance_reference = bert_score_result_vector_low_guidance_reference['f1'][0]

        cross_encoder_score_vector_low_guidance_reference = model.predict([[teacher_answer, response_vector_low_guidance]])[0]

        # High guidance.
        bleu_score_vector_high_guidance_reference = sentence_bleu([teacher_answer.split()], response_vector_high_guidance.split())

        bert_score_result_vector_high_guidance_reference = bert_score.compute(predictions=[response_vector_high_guidance], references=[teacher_answer], lang="en")
        bert_score_f_vector_high_guidance_reference = bert_score_result_vector_high_guidance_reference['f1'][0]

        cross_encoder_score_vector_high_guidance_reference = model.predict([[teacher_answer, response_vector_high_guidance]])[0]

        # Graph

        # No guidance.
        bleu_score_graph_no_guidance_reference = sentence_bleu([teacher_answer.split()], response_graph_no_guidance.split())

        bert_score_result_graph_no_guidance_reference = bert_score.compute(predictions=[response_graph_no_guidance], references=[teacher_answer], lang="en")
        bert_score_f_graph_no_guidance_reference = bert_score_result_graph_no_guidance_reference['f1'][0]

        cross_encoder_score_graph_no_guidance_reference = model.predict([[teacher_answer, response_graph_no_guidance]])[0]

        # Low guidance.
        bleu_score_graph_low_guidance_reference = sentence_bleu([teacher_answer.split()], response_graph_low_guidance.split())

        bert_score_result_graph_low_guidance_reference = bert_score.compute(predictions=[response_graph_low_guidance], references=[teacher_answer], lang="en")
        bert_score_f_graph_low_guidance_reference = bert_score_result_graph_low_guidance_reference['f1'][0]

        cross_encoder_score_graph_low_guidance_reference = model.predict([[teacher_answer, response_graph_low_guidance]])[0]

        # High guidance.
        bleu_score_graph_high_guidance_reference = sentence_bleu([teacher_answer.split()], response_graph_high_guidance.split())

        bert_score_result_graph_high_guidance_reference = bert_score.compute(predictions=[response_graph_high_guidance], references=[teacher_answer], lang="en")
        bert_score_f_graph_high_guidance_reference = bert_score_result_graph_high_guidance_reference['f1'][0]

        cross_encoder_score_graph_high_guidance_reference = model.predict([[teacher_answer, response_graph_high_guidance]])[0]

    else:
        # Vector

        # No guidance.
        bleu_score_vector_no_guidance_reference = np.nan
        bert_score_f_vector_no_guidance_reference = np.nan
        cross_encoder_score_vector_no_guidance_reference = np.nan

        # Low guidance.
        bleu_score_vector_low_guidance_reference = np.nan
        bert_score_f_vector_low_guidance_reference = np.nan
        cross_encoder_score_vector_low_guidance_reference = np.nan

        # High guidance.
        bleu_score_vector_high_guidance_reference = np.nan
        bert_score_f_vector_high_guidance_reference = np.nan
        cross_encoder_score_vector_high_guidance_reference = np.nan

        # Graph

        # No guidance.
        bleu_score_graph_no_guidance_reference = np.nan
        bert_score_f_graph_no_guidance_reference = np.nan
        cross_encoder_score_graph_no_guidance_reference = np.nan

        # Low guidance.
        bleu_score_graph_low_guidance_reference = np.nan
        bert_score_f_graph_low_guidance_reference = np.nan
        cross_encoder_score_graph_low_guidance_reference = np.nan

        # High guidance.
        bleu_score_graph_high_guidance_reference = np.nan
        bert_score_f_graph_high_guidance_reference = np.nan
        cross_encoder_score_graph_high_guidance_reference = np.nan

    # Vector

    # No guidance.
    if not pd.isna(context_vector_no_guidance):
        bleu_score_vector_no_guidance_context = sentence_bleu([context_vector_no_guidance.split()], response_vector_no_guidance.split())

        bert_score_result_vector_no_guidance_context = bert_score.compute(predictions=[response_vector_no_guidance], references=[context_vector_no_guidance], lang="en")
        bert_score_f_vector_no_guidance_context = bert_score_result_vector_no_guidance_context['f1'][0]

        cross_encoder_score_vector_no_guidance_context = model.predict([[context_vector_no_guidance, response_vector_no_guidance]])[0]
    else:
        bleu_score_vector_no_guidance_context = np.nan
        bert_score_f_vector_no_guidance_context = np.nan
        cross_encoder_score_vector_no_guidance_context = np.nan

    # Low guidance.
    if not pd.isna(context_vector_low_guidance):
        bleu_score_vector_low_guidance_context = sentence_bleu([context_vector_low_guidance.split()], response_vector_low_guidance.split())

        bert_score_result_vector_low_guidance_context = bert_score.compute(predictions=[response_vector_low_guidance], references=[context_vector_low_guidance], lang="en")
        bert_score_f_vector_low_guidance_context = bert_score_result_vector_low_guidance_context['f1'][0]

        cross_encoder_score_vector_low_guidance_context = model.predict([[context_vector_low_guidance, response_vector_low_guidance]])[0]
    else:
        bleu_score_vector_low_guidance_context = np.nan
        bert_score_f_vector_low_guidance_context = np.nan
        cross_encoder_score_vector_low_guidance_context = np.nan

    # High guidance.
    if not pd.isna(context_vector_high_guidance):
        bleu_score_vector_high_guidance_context = sentence_bleu([context_vector_high_guidance.split()], response_vector_high_guidance.split())

        bert_score_result_vector_high_guidance_context = bert_score.compute(predictions=[response_vector_high_guidance], references=[context_vector_high_guidance], lang="en")
        bert_score_f_vector_high_guidance_context = bert_score_result_vector_high_guidance_context['f1'][0]

        cross_encoder_score_vector_high_guidance_context = model.predict([[context_vector_high_guidance, response_vector_high_guidance]])[0]
    else:
        bleu_score_vector_high_guidance_context = np.nan
        bert_score_f_vector_high_guidance_context = np.nan
        cross_encoder_score_vector_high_guidance_context = np.nan

    # Graph.

    # No guidance.
    if not pd.isna(context_graph_no_guidance):
        bleu_score_graph_no_guidance_context = sentence_bleu([context_graph_no_guidance.split()], response_graph_no_guidance.split())

        bert_score_result_graph_no_guidance_context = bert_score.compute(predictions=[response_graph_no_guidance], references=[context_graph_no_guidance], lang="en")
        bert_score_f_graph_no_guidance_context = bert_score_result_graph_no_guidance_context['f1'][0]

        cross_encoder_score_graph_no_guidance_context = model.predict([[context_graph_no_guidance, response_graph_no_guidance]])[0]
    else:
        bleu_score_graph_no_guidance_context = np.nan
        bert_score_f_graph_no_guidance_context = np.nan
        cross_encoder_score_graph_no_guidance_context = np.nan

    # Low guidance.
    if not pd.isna(context_graph_low_guidance):
        bleu_score_graph_low_guidance_context = sentence_bleu([context_graph_low_guidance.split()], response_graph_low_guidance.split())

        bert_score_result_graph_low_guidance_context = bert_score.compute(predictions=[response_graph_low_guidance], references=[context_graph_low_guidance], lang="en")
        bert_score_f_graph_low_guidance_context = bert_score_result_graph_low_guidance_context['f1'][0]

        cross_encoder_score_graph_low_guidance_context = model.predict([[context_graph_low_guidance, response_graph_low_guidance]])[0]
    else:
        bleu_score_graph_low_guidance_context = np.nan
        bert_score_f_graph_low_guidance_context = np.nan
        cross_encoder_score_graph_low_guidance_context = np.nan

    # High guidance.
    if not pd.isna(context_graph_high_guidance):
        bleu_score_graph_high_guidance_context = sentence_bleu([context_graph_high_guidance.split()], response_graph_high_guidance.split())

        bert_score_result_graph_high_guidance_context = bert_score.compute(predictions=[response_graph_high_guidance], references=[context_graph_high_guidance], lang="en")
        bert_score_f_graph_high_guidance_context = bert_score_result_graph_high_guidance_context['f1'][0]

        cross_encoder_score_graph_high_guidance_context = model.predict([[context_graph_high_guidance, response_graph_high_guidance]])[0]
    else:
        bleu_score_graph_high_guidance_context = np.nan
        bert_score_f_graph_high_guidance_context = np.nan
        cross_encoder_score_graph_high_guidance_context = np.nan

    # Append results to their respective lists

    # Vector
    # no guidance reference
    metrics_results['bleu_score_vector_no_guidance_reference'].append(bleu_score_vector_no_guidance_reference)
    metrics_results['bert_score_f_vector_no_guidance_reference'].append(bert_score_f_vector_no_guidance_reference)
    metrics_results['cross_encoder_score_vector_no_guidance_reference'].append(cross_encoder_score_vector_no_guidance_reference)
    # low guidance reference
    metrics_results['bleu_score_vector_low_guidance_reference'].append(bleu_score_vector_low_guidance_reference)
    metrics_results['bert_score_f_vector_low_guidance_reference'].append(bert_score_f_vector_low_guidance_reference)
    metrics_results['cross_encoder_score_vector_low_guidance_reference'].append(cross_encoder_score_vector_low_guidance_reference)
    # high guidance reference
    metrics_results['bleu_score_vector_high_guidance_reference'].append(bleu_score_vector_high_guidance_reference)
    metrics_results['bert_score_f_vector_high_guidance_reference'].append(bert_score_f_vector_high_guidance_reference)
    metrics_results['cross_encoder_score_vector_high_guidance_reference'].append(cross_encoder_score_vector_high_guidance_reference)
    # no guidance context
    metrics_results['bleu_score_vector_no_guidance_context'].append(bleu_score_vector_no_guidance_context)
    metrics_results['bert_score_f_vector_no_guidance_context'].append(bert_score_f_vector_no_guidance_context)
    metrics_results['cross_encoder_score_vector_no_guidance_context'].append(cross_encoder_score_vector_no_guidance_context)
    # low guidance context
    metrics_results['bleu_score_vector_low_guidance_context'].append(bleu_score_vector_low_guidance_context)
    metrics_results['bert_score_f_vector_low_guidance_context'].append(bert_score_f_vector_low_guidance_context)
    metrics_results['cross_encoder_score_vector_low_guidance_context'].append(cross_encoder_score_vector_low_guidance_context)
    # high guidance context
    metrics_results['bleu_score_vector_high_guidance_context'].append(bleu_score_vector_high_guidance_context)
    metrics_results['bert_score_f_vector_high_guidance_context'].append(bert_score_f_vector_high_guidance_context)
    metrics_results['cross_encoder_score_vector_high_guidance_context'].append(cross_encoder_score_vector_high_guidance_context)

    # Graph
    # no guidance reference
    metrics_results['bleu_score_graph_no_guidance_reference'].append(bleu_score_graph_no_guidance_reference)
    metrics_results['bert_score_f_graph_no_guidance_reference'].append(bert_score_f_graph_no_guidance_reference)
    metrics_results['cross_encoder_score_graph_no_guidance_reference'].append(cross_encoder_score_graph_no_guidance_reference)
    # low guidance reference
    metrics_results['bleu_score_graph_low_guidance_reference'].append(bleu_score_graph_low_guidance_reference)
    metrics_results['bert_score_f_graph_low_guidance_reference'].append(bert_score_f_graph_low_guidance_reference)
    metrics_results['cross_encoder_score_graph_low_guidance_reference'].append(cross_encoder_score_graph_low_guidance_reference)
    # high guidance reference
    metrics_results['bleu_score_graph_high_guidance_reference'].append(bleu_score_graph_high_guidance_reference)
    metrics_results['bert_score_f_graph_high_guidance_reference'].append(bert_score_f_graph_high_guidance_reference)
    metrics_results['cross_encoder_score_graph_high_guidance_reference'].append(cross_encoder_score_graph_high_guidance_reference)
    # no guidance context
    metrics_results['bleu_score_graph_no_guidance_context'].append(bleu_score_graph_no_guidance_context)
    metrics_results['bert_score_f_graph_no_guidance_context'].append(bert_score_f_graph_no_guidance_context)
    metrics_results['cross_encoder_score_graph_no_guidance_context'].append(cross_encoder_score_graph_no_guidance_context)
    # low guidance context
    metrics_results['bleu_score_graph_low_guidance_context'].append(bleu_score_graph_low_guidance_context)
    metrics_results['bert_score_f_graph_low_guidance_context'].append(bert_score_f_graph_low_guidance_context)
    metrics_results['cross_encoder_score_graph_low_guidance_context'].append(cross_encoder_score_graph_low_guidance_context)
    # high guidance context
    metrics_results['bleu_score_graph_high_guidance_context'].append(bleu_score_graph_high_guidance_context)
    metrics_results['bert_score_f_graph_high_guidance_context'].append(bert_score_f_graph_high_guidance_context)
    metrics_results['cross_encoder_score_graph_high_guidance_context'].append(cross_encoder_score_graph_high_guidance_context)

# Now, create a new DataFrame from metrics_results
calc_df = merged_df_all.copy()
for metric, results in metrics_results.items():
    calc_df[metric] = results

# Save calc_df